<a href="https://colab.research.google.com/github/SaiSJoshi/Neural-Signal-Processing/blob/main/Exploratory_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data


In [125]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [126]:
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [127]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
%matplotlib inline
import SimpleITK as sitk


In [124]:
train_dir = '/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/Images/Trail1' # image folder
# get the list of jpegs from sub image class folders
# normal_images = []
# for fn in os.listdir(f'{train_dir}/NORMAL'):
#   if fn.endswith('.jpeg'):
#     normal_images.append(fn)
images_1_5T_path = r'/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/Images/Trail1/ADNI_1.5_T_1to15' # accessing image url
images_3T_path = r'/content/gdrive/MyDrive/AD_NET/AD-Net/sai_stuff/Images/Trail1/ADNI_3_T_1to15' # acessing image url
t_1_5_imgs = [fn for fn in os.listdir(f'{train_dir}/ADNI_1.5_T_1to15') if fn.endswith('.nii')]
t_3_imgs = [fn for fn in os.listdir(f'{train_dir}/ADNI_3_T_1to15') if fn.endswith('.nii')]
# t_3_imgs = [fn for fn in os.listdir(f'{images_3T_path}') if fn.endswith('.nii')]
print(t_1_5_imgs)

[]


# Image Matrix

In [121]:
# making n X m matrix
def img2np(path, list_of_filename, size = (64, 64)):
    # iterating through each file
    full_mat = None
    # print(list_of_filename)
    for fn in list_of_filename:
        img_path = path + fn
        
        # Read the image and print its size and display one slice algong with converting it into Numpy
        img = sitk.ReadImage(img_path)
        fp = sitk.GetArrayFromImage(img)#converting into Numpy
        print(fp.shape)
        print(img.GetSize())
        plt.imshow(fp[:,120,:])
        
        
        current_image = image.load_img(fp, target_size = size, color_mode = 'grayscale')

        
        # covert image to a matrix
        img_ts = image.img_to_array(current_image)
        
        # turn that into a vector / 1D array
        img_ts = [img_ts.ravel()]
        if full_mat:
            # concatenate different images
            full_mat = np.concatenate((full_mat, img_ts))
        else: 
            # if not assigned yet, assign one
            full_mat = img_ts
    return full_mat

    

# run it on our folders
image_matrix_1_5_T = img2np(f'{images_1_5T_path}/1.5T/', t_1_5_imgs)
# print(image_matrix_1_5_T, t_1_5_imgs)
image_matrix_3_T = img2np(f'{images_3T_path}/3T/', t_3_imgs)


[]
[]


# Average Image

In [107]:
def find_mean_img(full_mat, title, size = (64, 64)):
    
    # calculate the average
    mean_img = np.mean(full_mat, axis = 0)

    # reshape it back to a matrix
    mean_img = mean_img.reshape(size)
    plt.imshow(mean_img, vmin=0, vmax=255, cmap='Greys_r')
    plt.title(f'Average {title}')
    plt.axis('off')
    plt.show()
    return mean_img

mean_1_5_T = find_mean_img(image_matrix_1_5_T,'1.5_T')
mean_3_T = find_mean_img(image_matrix_3_T,'3_T')

AxisError: ignored

# Contrast between Average Images

In [ ]:
contrast_mean = norm_mean - AD_mean
plt.imshow(contrast_mean, cmap='bwr')
plt.title(f'Difference Between Normal & AD Average')
plt.axis('off')
plt.show()

# Eigen Images

In [ ]:
from sklearn.decomposition import PCA
from math import ceil

def eigenimages(full_mat, title, n_comp = 0.7, size = (64, 64)):
    # fit PCA to describe n_comp * variability in the class
    pca = PCA(n_components = n_comp, whiten = True)
    pca.fit(full_mat)
    print('Number of PC: ', pca.n_components_)
    return pca
  
def plot_pca(pca, size = (64, 64)):
    # plot eigenimages in a grid
    n = pca.n_components_
    fig = plt.figure(figsize=(8, 8))
    r = int(n**.5)
    c = ceil(n/ r)
    for i in range(n):
        ax = fig.add_subplot(r, c, i + 1, xticks = [], yticks = [])
        ax.imshow(pca.components_[i].reshape(size), 
                  cmap='Greys_r')
    plt.axis('off')
    plt.show()
    
plot_pca(eigenimages(normal_images, 'NORMAL'))
plot_pca(eigenimages(AD_images, 'AD'))